In [1]:
!pip install torch torchvision
!pip install tqdm
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [25]:
import pandas as pd
import numpy as np

import os
import os.path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

import glob
from PIL import Image

import numpy as np
from torchvision.models import resnet18, ResNet18_Weights, vgg11 , VGG11_Weights, resnet50, ResNet50_Weights,  mobilenet_v2, MobileNet_V2_Weights
from torchvision.transforms import (Compose,
                                    RandomResizedCrop,
                                    RandomHorizontalFlip,
                                    ToTensor,
                                    Normalize,
                                    Resize,
                                    CenterCrop
                                    )
import argparse
import shutil
import warnings
warnings.filterwarnings("ignore")

# Hyper parameter

Các hyper parameter mặc định cho việc train. Trong đó có một vài cái là đường dẫn file kaggle

In [26]:
def parse_arg():
    parse = argparse.ArgumentParser(description='Transfer learning')
    parse.add_argument('--epochs', '-e', type=int, default=100, help='Số lượng epoch để chạy')
    parse.add_argument('--checkpoint-dir', '-d', type=str, default='/kaggle/working/checkpoint', help='Nơi lưu checkpoint - weght của model')
    parse.add_argument('--tensorboard', '-t', type=str, default='/kaggle/working/dashboad', help='Nơi lưu file .event trực quan hóa')
    parse.add_argument('--lr', '-l', type=float, default=1e-3)
    parse.add_argument('--EarlyStopping','-s',type=int, default=70, help='Early stopping càng lớn thì train được càng nhiều')
    parse.add_argument('--batch-size','-b',type=int, default=16, help='Kích thước mỗi batch cho vào mô hình')
    args, unknown = parse.parse_known_args()
    return args


# Create DataLoader

Tiền xử lý dữ liệu bằng data loader

In [27]:
# Đường dẫn tới dataset trên kaggle (tùy chỉnh trước khi chạy)
temp_val = '/kaggle/input/aio-hutech/test'
temp_train = '/kaggle/input/aio-hutech/train'
test_path = "/kaggle/input/aio-hutech/submission.csv"

In [28]:
def preprocessing(dataset_path):
    transforms = Compose([
            RandomResizedCrop(224),
            RandomHorizontalFlip(),
            ToTensor(),
            Normalize([0.574, 0.574, 0.574], [0.169, 0.169, 0.169])
        ])

    dataset = ImageFolder(root=dataset_path, transform=transforms)
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=16,
        shuffle=True,
        num_workers=4,
        drop_last=True
    )

    return data_loader


# Function for evaluation

Đánh giá mô hình bằng metrics accuracy khi cần thiết

In [29]:
def metrics_evaluations(writer, optimizer, epoch, score, metrics, name, model,
                        model_checkpoint_path):
    writer.add_scalar(tag=f'{name}/val', scalar_value=score, global_step=epoch)

    # show kết quả

    checkpoint = {
        'state': model.state_dict(),
        'epoch': epoch,
        'optimizer': optimizer.state_dict(),
        'best_epoch': metrics['best_epoch'],
        f'best_{name}': metrics[f'best_{name}']
    }    
    
    if metrics[f'best_{name}'] < score:
        metrics[f'best_{name}'] = score
        metrics[f'best_epoch'] = epoch
        torch.save(checkpoint, model_checkpoint_path + '/' + f'best_{name}.pt')



# Transfer learning

**Tái huấn luyện mô hình**

In [30]:
def transfer_learning(model, model_name, classes, criterion, optimizer, parse, data_loader):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    metrics = {
                'best_accuracy' : -999,
                'best_epoch' : 0
    }
    # Create a SummaryWriter for TensorBoard
    writer = SummaryWriter(parse.tensorboard)
    
    for epoch in range(parse.epochs):
        print('-' * 10)
        print(f'Epoch {epoch + 1}/{parse.epochs}')

        # training
        model.train()
        loss_recorded = []
        all_pred = []
        all_labels = []

        progress_bar = tqdm(data_loader, colour='yellow')
        for iter, (images, labels) in enumerate(progress_bar):
            images = images.to(device)
            labels = labels.to(device)

            # foward
            pred = model(images)
            max_pred = torch.argmax(pred, dim=1)
            loss = criterion(pred, labels)

            # backward
            optimizer.zero_grad()
            loss.backward() 
            optimizer.step() 

            # ghi nhận loss
            all_labels.extend(labels.tolist())
            all_pred.extend(max_pred.tolist())
            loss_recorded.append(loss.item())

            progress_bar.set_description(f'Train/Batch {iter}, Loss: {loss.item():.4f}')
            writer.add_scalar(tag='Train/Loss', scalar_value=np.mean(loss_recorded),
                              global_step=epoch * len(data_loader) + iter)

        # numerical metrics
        acc = accuracy_score(all_labels, all_pred)
        writer.add_scalar(tag='Train/Accuracy', scalar_value=acc,
                          global_step=epoch * len(data_loader) + iter)
        print(f'Acc: {acc:.4f}, avg_oss: {np.mean(loss_recorded):.4f}')
        if abs(metrics['best_epoch'] - epoch) >= parse.EarlyStopping:
            break

    writer.close()

In [31]:
data = preprocessing(dataset_path=temp_train)
parse = parse_arg()
classes = ['nấm mỡ', 'bào ngư xám + trắng', 'Đùi gà Baby (cắt ngắn)', 'linh chi trắng']

# criterion
criterion = nn.CrossEntropyLoss()

# Một vài chiến lược cải thiện mô hình bằng hyper parameters

- Early stopping
  + Càng nhỏ thì học càng nhanh, nhưng độ chính xác không cao
  + Càng lớn thì độ chính xác cao nhưng mô hình học càng chậm,

- learning rate
  + Càng nhỏ thì càng tối ưu nhưng chạy chậm
  + Càng lớn thì ngược lại

- Batch size
  + Càng nhỏ: Tiêu tốn ít RAM hơn, tổng quát hóa tốt hơn, nhưng chậm
  + Càng lớn: Tiêu tốn nhiều RAM hơn, dễ bị overfit
  
- optimizer
  + Adam
  + SGD
  + Adagrad
  + momemtum
    
- Đổi mô hình:
    + Mô hình càng lớn, dữ liệu càng nhỏ thì dễ bị overfiting

      Giải pháp: chọn mô hình nhỏ hơn để phù hợp với dữ liệu

# Chạy thử với mô hình mobilenet_v3_small

In [32]:
# --------------------------------- mobilenet_v3_small ---------------------------------
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
mobilenet = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights)
mobilenet.classifier[3] = nn.Linear(in_features=1024, out_features=len(classes), bias=True)

optimizer = optim.SGD(mobilenet.parameters(),momentum=0.9, lr=parse.lr, weight_decay=5e-4)

transfer_learning(mobilenet,
                    'mobilenet_v3_small',
                    classes,
                    criterion,
                    optimizer,
                    parse,
                    data
                    )

----------
Epoch 1/100


Train/Batch 74, Loss: 0.6635: 100%|██████████| 75/75 [00:03<00:00, 24.50it/s]


Acc: 0.6100, avg_oss: 1.0651
----------
Epoch 2/100


Train/Batch 74, Loss: 0.7705: 100%|██████████| 75/75 [00:03<00:00, 23.31it/s]


Acc: 0.8183, avg_oss: 0.5781
----------
Epoch 3/100


...

Acc: 0.9717, avg_oss: 0.0834
----------
Epoch 70/100


Train/Batch 74, Loss: 0.0337: 100%|██████████| 75/75 [00:03<00:00, 24.69it/s]


Acc: 0.9667, avg_oss: 0.0894
----------
Epoch 71/100


Train/Batch 74, Loss: 0.0087: 100%|██████████| 75/75 [00:03<00:00, 24.22it/s]


Acc: 0.9708, avg_oss: 0.0912


# ***Predict***

In [33]:
# Custom dataset for flat directory structure
class TestImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")) + 
                                  glob.glob(os.path.join(image_dir, "*.jpeg")) + 
                                  glob.glob(os.path.join(image_dir, "*.png")))
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        # Return image and path for identification
        return image, os.path.basename(img_path)

In [34]:
# def predict_test(model_path, test_dir=None, batch_size=20, device=None):
#     """
#     Predict and evaluate a trained model on test data using classification_report
    
#     Args:
#         model_path (str): Path to the saved model checkpoint
#         test_dir (str): Directory containing test images (flat structure, no class folders)
#         batch_size (int): Batch size for inference
#         device (torch.device): Device to run inference on (None for auto-detection)
        
#     Returns:
#         dict: Classification report metrics (if ground truth is available)
#     """
#     # Set default directories if not provided
#     if test_dir is None:
#         test_dir = '/kaggle/input/aio-hutech/test'
    
#     # Auto-detect device if not specified
#     if device is None:
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")

#     class_names = classes

#     # Load test data
#     test_transforms = Compose([
#             RandomResizedCrop(224),
#             RandomHorizontalFlip(),
#             ToTensor(),
#             Normalize([0.574, 0.574, 0.574], [0.169, 0.169, 0.169])
#         ])

#     # Create test dataset with our custom class
#     test_dataset = TestImageDataset(test_dir, transform=test_transforms)
    
#     if len(test_dataset) == 0:
#         raise FileNotFoundError(f"No image files found in {test_dir}")
#     print(f"Found {len(test_dataset)} test images")
    
#     test_loader = DataLoader(
#         dataset=dataset,
#         batch_size=20,
#         shuffle=True,
#         num_workers=4
#     )

#     # Load the model
#     model = mobilenet

#     # Run inference
#     all_preds = []
#     all_filenames = []
    
#     print("Running inference on test data...")
#     with torch.no_grad():
#         for images, filenames in tqdm(test_loader, desc="Evaluating"):
#             images = images.to(device)
            
#             outputs = model(images)
#             _, preds = torch.max(outputs, 1)
            
#             all_preds.extend(preds.cpu().numpy())
#             all_filenames.extend(filenames)

In [ ]:
test_dir = temp_val
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
class_names = classes
model = mobilenet

In [35]:
# Load test data
test_transforms = Compose([
        RandomResizedCrop(224),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize([0.574, 0.574, 0.574], [0.169, 0.169, 0.169])
    ])

# Create test dataset with our custom class
test_dataset = TestImageDataset(test_dir, transform=test_transforms)

if len(test_dataset) == 0:
    raise FileNotFoundError(f"No image files found in {test_dir}")
print(f"Found {len(test_dataset)} test images")

test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=20,
        shuffle=True,
        num_workers=4
    )

Using device: cuda
Found 200 test images
Running inference on test data...


Evaluating: 100%|██████████| 10/10 [00:00<00:00, 17.01it/s]


In [ ]:
# Run inference
all_preds = []
all_filenames = []

In [ ]:
print("Running inference on test data...")
with torch.no_grad():
    for images, filenames in tqdm(test_loader, desc="Evaluating"):
        images = images.to(device)
        
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_filenames.extend(filenames)

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame({
    'filename': [os.path.splitext(filename)[0] for filename in all_filenames],
    'type': all_preds
    # 'predicted_class': [class_names[idx] for idx in all_preds]
}).sort_values(by='filename')

In [36]:
test_df = pd.read_csv(test_path)

In [37]:
print(classification_report(test_df["type"], results_df["type"]))

              precision    recall  f1-score   support

           0       0.21      0.24      0.23        50
           1       0.30      0.34      0.32        50
           2       0.24      0.26      0.25        50
           3       0.28      0.18      0.22        50

    accuracy                           0.26       200
   macro avg       0.26      0.26      0.25       200
weighted avg       0.26      0.26      0.25       200



In [38]:
# Save predictions to CSV
csv_path = 'submission.csv'
results_df.to_csv(csv_path, index=False)
print(f"Saved predictions to {csv_path}")

Saved predictions to submission.csv
